In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import StaleElementReferenceException



import datetime


# options = Options()
# options.add_argument("--start-maximized")  # Maximize the browser window
# options.add_argument("--disable-extensions")
# options.add_argument("--disable-popup-blocking")
# options.add_argument("--disable-infobars")
# options.add_argument("--no-sandbox")
# options.add_argument("--disable-dev-shm-usage")
# options.add_argument("--disable-gpu")
# options.add_argument("--ignore-certificate-errors")

#driver = webdriver.Chrome()
#driver = webdriver.Safari()

ERROR = False

cookies_ec_dict = {'google': EC.element_to_be_clickable((By.XPATH, "//div[@class='QS5gu sy4vM']")),
                'nasdaq': EC.element_to_be_clickable((By.ID, "onetrust-accept-btn-handler")),
                'investing.com':EC.element_to_be_clickable((By.ID, "onetrust-accept-btn-handler")),
                'zacks.com_readmore': EC.visibility_of_element_located((By.XPATH, "//button[contains(@class, 'Button__StyledButton') and contains(text(), 'Read more to accept preferences')]")),
                'zacks.com': EC.element_to_be_clickable((By.XPATH, "//button[contains(@class, 'Button__StyledButton-a1qza5-0') and contains(text(), 'Accept all')]"))}

nav_nasdaq_ec_dict = {'select_date': EC.element_to_be_clickable((By.ID, "date-picker__toggle")),
                            #"//button[@class='date-picker__toggle' and @aria-label='click show datepicker popup']")),      
                      'select_searchbox':  EC.visibility_of_element_located((By.CLASS_NAME, "date-picker__input")),
                      'apply_date_button': EC.element_to_be_clickable((By.XPATH,
                            "//button[@class='date-picker__apply']"))}

nav_investingcom_ec_dict = {'select_calendar': EC.element_to_be_clickable((By.ID, "datePickerToggleBtn"))}

def accept_cookies(site = 'google'):   
    if site == 'zacks.com':
        read_more = WebDriverWait(driver, 10).until(cookies_ec_dict['zacks.com_readmore']) 
        read_more.click()
    try:
        accept = WebDriverWait(driver, 1).until(cookies_ec_dict[site])
        accept.click()
    except:
        print("No 'Accept' button found or unable to click on ", site)


def google(query, cookies):
    # Open the website
    driver.get("https://google.com")
    if cookies: accept_cookies('google')
    search_box = driver.find_element(By.NAME, "q")

     # Enter your search query
    search_query = query
    search_box.send_keys(search_query)

    # Simulate hitting Enter
    search_box.send_keys(Keys.RETURN)

def click_google_site(site):
    driver.find_element(By.XPATH, f"//a[contains(@href, '{site}')]").click()

def select_date(year, month, day, site = 'nasdaq'):
    if site == 'nasdaq':
        date = datetime.date(year, month, day).strftime('%m/%d/%Y')
        WebDriverWait(driver, 1).until(nav_nasdaq_ec_dict['select_date']).click()
        driver.execute_script(f"document.querySelector('.date-picker__input').value = '{date}';")
        WebDriverWait(driver, 1).until(nav_nasdaq_ec_dict['e_button']).click()
    elif site == 'investing.com':
        date = datetime.date(year, month, day).strftime('%m/%d/%Y')
        WebDriverWait(driver, 1).until(nav_investingcom_ec_dict['select_calendar']).click()
        input_element = driver.find_element(By.CSS_SELECTOR, "input.newInput#startDate")
        input_element.clear()  # Clear any existing value
        input_element.send_keys(date)  
        input_element = driver.find_element(By.CSS_SELECTOR, "input.newInput#endDate")
        input_element.clear()  # Clear any existing value
        input_element.send_keys(date)  
        driver.find_element(By.ID, 'applyBtn').click()
            
def google_site(query, site, cookies):
    google(query, cookies)
    click_google_site(site)


def get_symbols(site):
    symbols = []
    if site == 'investing.com':
        table = driver.find_element(By.ID, "earningsCalendarData")
        rows = table.find_elements(By.TAG_NAME, "tr")
        # Iterate over each table row
        for row in rows:
            try:
                # Find all table cells (td elements) within the row
                cells = row.find_elements(By.TAG_NAME, "td")
                # Extract and print the text content of each cell
                row_data = [cell.text for cell in cells]
            except StaleElementReferenceException:
                # Retry finding the table row
                row = driver.find_element(By.ID, "earningsCalendarData").find_element(By.TAG_NAME, "tr")
                cells = row.find_elements(By.TAG_NAME, "td")
                row_data = [cell.text for cell in cells]
            if len(row_data) > 1:
                company = row_data[1]
                symbols.append(company[company.find('(')+1:company.find(')')])

    return symbols

def search_on(website, query):
    if website == 'zacks.com':
        search_input = WebDriverWait(driver, 20).until(
            EC.visibility_of_element_located((By.ID, "search-q"))
        )
        # Input the search query
        search_input.send_keys(query)
        # Press ENTER to submit the form
        search_input.send_keys(Keys.ENTER)


def get_rank(website):
    if website == 'zacks.com':
        try:
            # Wait for the paragraph element to be visible
            rank_paragraph = WebDriverWait(driver, 20).until(
                EC.visibility_of_element_located((By.CLASS_NAME, "rank_view"))
            )

            # Get the text of the paragraph element
            rank_text = rank_paragraph.text.strip()
            return int(rank_text[0])
        
        except Exception as e:
            ERROR = True

            
def analyst(website, symbol, cookies):
    if website == 'zacks.com':
        driver.get(f'https://www.zacks.com/stock/quote/{symbol}?q={symbol}')
    else:
        google_site(f'{website[:website.find(".")]} {symbol}', website, cookies)
    if cookies: accept_cookies(website)
    rank = get_rank(website)
    return rank
   
def earnings_rank(year, month, day):
    symbol_rank = {}
    google_site('earnings calendar', 'investing.com', cookies = True)
    accept_cookies('investing.com')
    select_date(year, month, day, site = 'investing.com')
    symbols = get_symbols('investing.com')
    for index, symbol in enumerate(symbols):
        print(symbol)
        rank = analyst('zacks.com', symbol, cookies = not(index))
        symbol_rank[symbol] = rank

    return symbol_rank



##########################################


In [65]:
driver = webdriver.Chrome()
year, month, day = 2024, 3, 5
earnings_rank(year, month, day)

CRWD
True
opa
TGT
False
ROST
False
FERG
False
JD
False
ASHTY
False
BAYRY
False
FNV
False
NIO
False
RADLY
False
AS
False
NDBKY
False
HCP
False
HCXLF
False
FNLPF
False
GGAL
False
BOX
False
NVEI
False
JWN
False
SBSW
False
GBTG
False
ODD
False
LU
False
BBAR
False
FWRG
False
MLNK
False
BASE
False
AGTI
False
CDRE
False
SEAT
False
VVX
False
SRRK
False
CRCT
False
CHPT
False
RSKD
False
CECO
False
CIFR
False
VTOL
False
MYE
False
SMRT
False
DADA
False
LXU
False
MRNS
False
YORW
False
CNSL
False
SD
False
OFIX
False
ALNT
False
AXGN
False
WTI
False
GLRE
False
NPCE
False
NYXH
False
VNET
False
CERS
False
AGS
False
EVC
False
PKOH
False
SMLR
False
SOPH
False
WEYS
False
TSVT
False
AQST
False
NEWT
False
RIGL
False
BKCC
False
FSTR
False
RNGR
False
AOMR
False
OOMA
False
MEC
False
MASS
False
PLCE
False
CGEN
False
AEVA
False
CVGI
False
APEI
False
NVCT
False
FLL
False
MCRB
False
OFS
False
XTNT
False
CXDO
False
BW
False
SKYX
False
ACNT
False
MPU
False
STIM
False
FACO
False
APT
False
RCAT
False
BWEN
False
LQMT
Fa

{'CRWD': 1,
 'TGT': 3,
 'ROST': 3,
 'FERG': 3,
 'JD': 3,
 'ASHTY': 4,
 'BAYRY': 4,
 'FNV': 3,
 'NIO': 2,
 'RADLY': None,
 'AS': 3,
 'NDBKY': None,
 'HCP': 2,
 'HCXLF': None,
 'FNLPF': 4,
 'GGAL': 4,
 'BOX': 3,
 'NVEI': 3,
 'JWN': 3,
 'SBSW': 3,
 'GBTG': 3,
 'ODD': 2,
 'LU': 5,
 'BBAR': 3,
 'FWRG': 4,
 'MLNK': 2,
 'BASE': 3,
 'AGTI': 4,
 'CDRE': 2,
 'SEAT': 4,
 'VVX': 4,
 'SRRK': 3,
 'CRCT': 3,
 'CHPT': 4,
 'RSKD': 3,
 'CECO': 3,
 'CIFR': 3,
 'VTOL': None,
 'MYE': 3,
 'SMRT': 4,
 'DADA': 3,
 'LXU': 3,
 'MRNS': 3,
 'YORW': 3,
 'CNSL': None,
 'SD': None,
 'OFIX': 3,
 'ALNT': 3,
 'AXGN': 2,
 'WTI': 5,
 'GLRE': None,
 'NPCE': 3,
 'NYXH': 3,
 'VNET': 3,
 'CERS': 4,
 'AGS': 3,
 'EVC': 3,
 'PKOH': 3,
 'SMLR': None,
 'SOPH': 3,
 'WEYS': None,
 'TSVT': 3,
 'AQST': 3,
 'NEWT': 3,
 'RIGL': 3,
 'BKCC': 4,
 'FSTR': 3,
 'RNGR': 5,
 'AOMR': 3,
 'OOMA': 3,
 'MEC': 3,
 'MASS': 3,
 'PLCE': 4,
 'CGEN': 2,
 'AEVA': 3,
 'CVGI': 3,
 'APEI': 2,
 'NVCT': 3,
 'FLL': None,
 'MCRB': 3,
 'OFS': 3,
 'XTNT': None,
 

In [2]:
import yfinance as yf

In [112]:
ticker = yf.Ticker('DELL')

In [116]:
hist = ticker.history()
hist

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2024-02-02 00:00:00-05:00,85.629997,86.900002,85.220001,86.320000,3683600,0.0,0.0
2024-02-05 00:00:00-05:00,86.529999,86.870003,84.760002,85.709999,2204200,0.0,0.0
2024-02-06 00:00:00-05:00,85.730003,85.989998,82.160004,82.680000,5013800,0.0,0.0
2024-02-07 00:00:00-05:00,83.440002,83.830002,82.779999,83.739998,2964200,0.0,0.0
2024-02-08 00:00:00-05:00,83.610001,85.139999,83.510002,84.800003,2993800,0.0,0.0
2024-02-09 00:00:00-05:00,85.250000,86.650002,85.010002,86.199997,2315700,0.0,0.0
2024-02-12 00:00:00-05:00,86.500000,86.889999,85.830002,86.620003,2979100,0.0,0.0
2024-02-13 00:00:00-05:00,84.989998,85.190002,83.449997,83.980003,3144300,0.0,0.0
2024-02-14 00:00:00-05:00,85.059998,86.230003,84.720001,86.040001,2361300,0.0,0.0


In [115]:
hist

<bound method log_indent_decorator.<locals>.wrapper of yfinance.Ticker object <DELL>>

In [94]:
hist['Low']['2024-02-02']

85.22000122070312

In [95]:
driver = webdriver.Chrome()


In [96]:
driver.quit()

In [10]:
ticker = yf.download(tickers="DELL", period="5d", interval="1m", prepost = True)



[*********************100%%**********************]  1 of 1 completed


In [11]:
# start = datetime.datetime(2021, 10, 18, 9, 30, 0)
# end = datetime.datetime(2021, 10, 18, 10, 30, 0)

# filtered = ticker[start: end]
# filtered

ticker['2024-03-01']

/var/folders/9g/bzb1c3hn6jvdx57km19ky1y00000gn/T/ipykernel_11591/453447690.py:7: FutureWarning: Indexing a DataFrame with a datetimelike index using a single string to slice the rows, like `frame[string]`, is deprecated and will be removed in a future version. Use `frame.loc[string]` instead.
  ticker['2024-03-01']


,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2024-03-01 04:00:00-05:00,114.000,114.93,113.0000,114.11,114.11,0
2024-03-01 04:01:00-05:00,114.480,114.82,114.4700,114.82,114.82,0
2024-03-01 04:02:00-05:00,114.800,115.00,114.5700,114.60,114.60,0
2024-03-01 04:03:00-05:00,114.600,114.79,114.1100,114.79,114.79,0
2024-03-01 04:04:00-05:00,114.770,115.00,114.7400,114.91,114.91,0
...,...,...,...,...,...,...
2024-03-01 19:55:00-05:00,124.190,124.20,124.1492,124.15,124.15,0
2024-03-01 19:56:00-05:00,124.150,124.25,124.1100,124.19,124.19,0
2024-03-01 19:57:00-05:00,124.200,124.20,124.0000,124.00,124.00,0
